DATA Breakdown: PS4 Games
https://www.kaggle.com/datasets/shivamb/all-playstation-4-games?resource=download

### Introduction:

In [35]:
#General import statements
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [36]:
#Read in the csv, and look at the columns, and observe what the columns alone might tell us and how it seems important
df = pd.read_csv(r"C:\Users\mbuck\OneDrive\Documents\CS549_Project\CS549_Project_CSV_Cleanup\playstation_4_games.csv")

df.columns.unique()

Index(['ID', 'GamePSID', 'GameName', 'Publisher', 'ReleaseYear', 'ReleaseDate',
       'Developer', 'Genre', 'Features', 'Size', 'Medium', 'Hardware',
       'CompletionTime(Hours)', 'is_Digital_game', 'is_Physical_game',
       'OfficialWebsite'],
      dtype='object')

In [37]:
df.head(10)
#Looking at these columns, I can infer that:
#ID is just referencing the order for the csv and not relevant for the end data, just for data manipulation
#GamePSID might be useful to keep if it seems that it has any relation to it's ID in the PS store?
#GameName is obviously important to know what title we're refrencing
#Publisher looks to be important if we want to recommend games with the same publisher
#ReleaseYear doesn't seem super important, but maybe if someone only buys older or newer games? 
#ReleaseDate is too specific, might interrupt data, it's probably realistic to assume that someone would buy a game based off a month or day, but year is okay.
#Genre, probably the most important metric for our data to evaluate
#Features, probably not super important, the first few entries seem to show us that it might be good, if people only want a game if it's within PS Now data or PS Plus
#Size, size of the same, probably not an important metric, but it could be?
#Medium, could be important if some people only buy digital only or avoid games that are digital only, probably not the best final metric but not a bad metric in general
#Hardware, could be useful, but maybe not? Will probably drop to avoid overfitting
#ComletionTime(Hours), a decent metric, maybe someone only likes indie games...Might be too specific, candidate for removal
#is_Digital_game, will have to observe if it produces better data than just Medium
#is_Physical_game, see above
#OfficialWebstie, tells us the website. Far from a useful metric. Will drop

,ID,GamePSID,GameName,Publisher,ReleaseYear,ReleaseDate,Developer,Genre,Features,Size,Medium,Hardware,CompletionTime(Hours),is_Digital_game,is_Physical_game,OfficialWebsite
0,1,3045,Q*Bert Rebooted,LOOT Interactive,2015.0,2015-02-17,Gonzo Games,Action,"PS Plus, PS Now",109.00,NaN,NaN,NaN,0,0,http://www.sidelineamusements.com/qbert/
1,2,3051,Q*Bert Rebooted (EU),LOOT Interactive,2015.0,2015-07-22,LOOT Interactive,Action,PS Now,127.00,Digital only,NaN,NaN,0,0,http://www.sidelineamusements.com/qbert/
2,3,6703,Q.U.B.E. 2,Trapped Nerve Games,2018.0,2018-03-13,Toxic Games,"Puzzle, Platformer",NaN,2682.88,Digital only,NaN,NaN,0,0,http://qube-game.com/
3,4,3403,Q.U.B.E. Director's Cut,GRIP Digital,2015.0,2015-07-21,Toxic Games,"Puzzle, Platformer",PS Plus,1945.60,Digital only,NaN,3-4,0,0,http://qube-game.com/
4,5,13176,Quake (PS4),Bethesda Softworks,2021.0,2021-08-19,"MachineGames, Nightdive Studios, id Software",First Person Shooter,NaN,NaN,Physical and Digital,NaN,NaN,0,0,https://bethesda.net/en/game/quake
5,6,-2913,Quantic Pinball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,https://bethesda.net/en/game/quake
6,7,-4119,Quantum Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,https://quantumerror.games/
7,8,-4106,Quantum League,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,https://quantum-league.com/
8,9,12156,Quantum Replica,PQube,2021.0,2021-05-14,ON3D Studios,Stealth,NaN,NaN,Digital only,PS4 Pro,NaN,0,0,http://pqube.co.uk/quantum-replica/
9,10,12157,Quantum Replica (EU),PQube,2021.0,2021-05-14,ON3D Studios,Stealth,NaN,NaN,Digital only,PS4 Pro,NaN,0,0,http://pqube.co.uk/quantum-replica/


In [38]:
df.info() #Doesn't show much, but nice to see non-null counts. At glance it seems HardWare, Features and CompletionTime(Hours) might be too small to keep. 
df.describe(include='all') 
#I see non-unique game names, 15 to be exact, will inspect these first and remove duplicate data
#1723 Unique Publishers, small number, but, seems very good to have for reccomendations!
#482 Genres! That's a lot, I'll take a look to see if any of these genres are duplicated as well. (i.e. any typos or spelling errors
#Why is ReleaseYear a float? Change to int.
#Those are the big observations for now.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9686 entries, 0 to 9685
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     9686 non-null   int64  
 1   GamePSID               9686 non-null   int64  
 2   GameName               9686 non-null   object 
 3   Publisher              8964 non-null   object 
 4   ReleaseYear            8952 non-null   float64
 5   ReleaseDate            8952 non-null   object 
 6   Developer              8987 non-null   object 
 7   Genre                  8898 non-null   object 
 8   Features               1749 non-null   object 
 9   Size                   7666 non-null   float64
 10  Medium                 8937 non-null   object 
 11  Hardware               1512 non-null   object 
 12  CompletionTime(Hours)  2230 non-null   object 
 13  is_Digital_game        9686 non-null   int64  
 14  is_Physical_game       9686 non-null   int64  
 15  Offi

,ID,GamePSID,GameName,Publisher,ReleaseYear,ReleaseDate,Developer,Genre,Features,Size,Medium,Hardware,CompletionTime(Hours),is_Digital_game,is_Physical_game,OfficialWebsite
count,9686.000000,9686.000000,9686,8964,8952.000000,8952,8987,8898,1749,7.666000e+03,8937,1512,2230,9686.0,9686.0,8089
unique,NaN,NaN,9671,1723,NaN,1659,3110,482,25,NaN,3,33,23,NaN,NaN,6126
top,NaN,NaN,Prince of Persia: The Sands of Time Remake,eastasiasoft,NaN,2021-09-30,Hamster,Platformer,Cross-Buy,NaN,Digital only,PS4 Pro,0-1,NaN,NaN,http://www.hamster.co.jp/american_hamster/arca...
freq,NaN,NaN,3,503,NaN,35,124,736,775,NaN,5927,748,802,NaN,NaN,17
mean,4843.500000,7450.627813,NaN,NaN,2018.515304,NaN,NaN,NaN,NaN,5.207188e+03,NaN,NaN,NaN,0.0,0.0,NaN
std,2796.251688,4434.041993,NaN,NaN,2.014025,NaN,NaN,NaN,NaN,1.880542e+04,NaN,NaN,NaN,0.0,0.0,NaN
min,1.000000,-5590.000000,NaN,NaN,2009.000000,NaN,NaN,NaN,NaN,1.100000e+01,NaN,NaN,NaN,0.0,0.0,NaN
25%,2422.250000,5108.250000,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN,3.922500e+02,NaN,NaN,NaN,0.0,0.0,NaN
50%,4843.500000,7972.500000,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN,1.392640e+03,NaN,NaN,NaN,0.0,0.0,NaN
75%,7264.750000,10749.750000,NaN,NaN,2020.000000,NaN,NaN,NaN,NaN,4.710400e+03,NaN,NaN,NaN,0.0,0.0,NaN


#### Looking at Duplicate Titles:

In [39]:
duplicate_df = df[df['GameName'].duplicated()]

duplicate_df.count()

duplicate_df.head(15)

,ID,GamePSID,GameName,Publisher,ReleaseYear,ReleaseDate,Developer,Genre,Features,Size,Medium,Hardware,CompletionTime(Hours),is_Digital_game,is_Physical_game,OfficialWebsite
868,869,13856,Riders Republic (PS4),Ubisoft,2021.0,2021-10-28,Ubisoft Annecy,"Sports, Cycling, Skiing, Snowboarding",Cross-Buy,NaN,Physical and Digital,PS4 Pro,NaN,0,0,https://www.ubisoft.com/en-us/game/riders-repu...
987,988,12823,Rogue Wizards,Spellbind Studios,NaN,NaN,Spellbind Studios,"Roguelite, Role Playing, Turn Based",NaN,NaN,Digital only,NaN,NaN,0,0,http://roguewizards.com/
988,989,12824,Rogue Wizards,Spellbind Studios,NaN,NaN,Spellbind Studios,"Roguelite, Role Playing, Turn Based",NaN,NaN,Digital only,NaN,NaN,0,0,http://roguewizards.com/
2463,2464,13199,OctaFight,Ghislain Avrillon,2021.0,2021-08-30,Pixel Almost Perfect,Action,NaN,NaN,Digital only,NaN,NaN,0,0,http://octafight.com/
2563,2564,13208,One-Eyed Lee and the Dinner Party (PS4),Ratalaika Games,2021.0,2021-08-27,DarkChibiShadow,Visual Novel,Cross-Buy,NaN,Digital only,NaN,0-1,0,0,https://www.ratalaikagames.com/games/oneeyedle...
3048,3049,11907,Prince of Persia: The Sands of Time Remake,Ubisoft,NaN,NaN,"Ubisoft Pune, Ubisoft Mumbai",Action-Adventure,NaN,NaN,Physical and Digital,NaN,NaN,0,0,https://www.ubisoft.com/en-us/game/prince-of-p...
3049,3050,11908,Prince of Persia: The Sands of Time Remake,Ubisoft,NaN,NaN,"Ubisoft Pune, Ubisoft Mumbai",Action-Adventure,NaN,NaN,Physical and Digital,NaN,NaN,0,0,https://www.ubisoft.com/en-us/game/prince-of-p...
4361,4362,12891,Slide Stars,NaN,NaN,NaN,Triangle Studios,Platformer,NaN,NaN,Digital only,NaN,NaN,0,0,NaN
4538,4539,12889,Speed 3: Grand Prix,NaN,NaN,NaN,Lion Castle Entertainment,"Automobile, Arcade Racing",NaN,NaN,Digital only,NaN,NaN,0,0,NaN
4539,4540,11003,Speed 3: Grand Prix,"Lion Castle Entertainment, GS2 Games, Mindscape",2020.0,2020-11-10,Lion Castle Entertainment,"Automobile, Arcade Racing",NaN,4730.88,Physical and Digital,NaN,NaN,0,0,https://gs2games.com/speed-3-grand-prix


In [40]:
#Here I'm looking at each individual piece of data from the above list from duplicates, looking at each title to determine which to drop, or if they're valid to keep as duplicates (I doubt it)

#df[df['GameName'] == "Riders Republic (PS4)"].head() #DROP ID = 868, both safe
#df[df['GameName'] == "Rogue Wizards"].head() #DROP ID = 988 and 989, only 987 has a Release Date so keep
#df[df['GameName'] == "OctaFight"].head() #Either are fine to keep, they share the exact same data, DROP ID = 2464
#df[df['GameName'] == "One-Eyed Lee and the Dinner Party (PS4)"].head() #Same as above, DROP ID = 2564, both entries have exact data
df[df['GameName'] == "Prince of Persia: The Sands of Time Remake"].head() #DROP ID = 11907 and 11908 for sake of the same issue above. Potentially drop 11906 also, for the separate csv that has only games that are released
#df[df['GameName'] == "Slide Stars"].head() #DROP ID = 4362, far less data than that of 4361
#df[df['GameName'] == "Speed 3: Grand Prix"].head() #DROP ID = 4538 and 4539, these two do not have release data or a publisher, and thus should not skew data so remove
#df[df['GameName'] == "Gunhouse"].head() #DROP ID = 6442 Has no info, looks to be for VR only? Keep 6443
#df[df['GameName'] == "Lust for Darkness"].head() #DROP ID = 7420, has no info for anything other than name and website. KEEP 7421
#df[df['GameName'] == "Conan Chop Chop"].head() #These are exactly the same, thus DROP ID = 10231
#df[df['GameName'] == "Crazy Athletics"].head() #DROP ID = 7918, Only difference is no genre
#df[df['GameName'] == "Klang 2 (JP) (PS4)"].head() #DROP ID = 7041,HOWEVER, No Genre, Probably won't keep either

drop_ids = [868, 988, 989, 2464, 2564, 11907, 11908, 4362, 4538, 4539, 6442, 7420, 10231, 7918, 7041]
#Observation, the titles with a negative valued GamePSID seems to be the ones with absolutely no data in it other than title, will test this in the next box
#Also, adding ids to drop within a list, to make it easier


In [41]:
test_df = df[df['GamePSID'] < 0]
test_df.count()
#This seems that there are 0 games with a genre, developer, or publisher mentioned

ID                       699
GamePSID                 699
GameName                 699
Publisher                  0
ReleaseYear                0
ReleaseDate                0
Developer                  0
Genre                      0
Features                   0
Size                       0
Medium                     0
Hardware                  14
CompletionTime(Hours)      0
is_Digital_game          699
is_Physical_game         699
OfficialWebsite          653
dtype: int64

In [42]:
test_df['Genre'].unique()
#I believe that this is safe to conclude, it's also safe to drop any game with a GamePSID < 0 as none of the entries in that category have a value other than NaN

array([nan], dtype=object)

Cleaning Up Our Data:

In [43]:
#Drop the data from our original dataframe who has a GamePSID value <=0 (or rather, keep GamePSID values that are > 0),
dropped_data_df = df[df['GamePSID'] > 0]
dropped_data_df = dropped_data_df[~dropped_data_df['ID'].isin(drop_ids)]

#Drop columns that dont seem to impact as much
dropped_data_df = dropped_data_df.drop('OfficialWebsite', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('ReleaseDate', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('Features', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('Hardware', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('CompletionTime(Hours)', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('Size', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('is_Digital_game', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('is_Physical_game', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.drop('GamePSID', axis = 1, inplace = False)
dropped_data_df = dropped_data_df.dropna(subset = ['Genre'])                                        #Willing to remove this line if necessary

dropped_data_df.info()
dropped_data_df.describe(include='all')
#Uh-Oh, there seems to be a couple of areas that still have data missing for Genre. 87 entries do not have any genre, but now we safely parsed and cleaned duplicate GameNames, we will probably purge this as well
#I also decided to remove the columns of Features, Hardware and CompletionTime because there were too little non-null entries, it may impact training data negatively. [Counts for them: 1746, 1497, 2229. Respectively]
#This means that, only [19.45%, 16.68%, and 24.83%] of the remaining data had information in their respective column. This may sway training data, and also does not provide a lot of information
#I chose to also drop ReleaseDate as Release Year seems to be a better metric that wont impact data as opposed to the full Date, I dropped OfficialWebsite because it had no impact on our data
#I also chose to drop Size, as I felt it wasn't very relevant for most people. The last columns, i.e. is_Digital_game and is_Physical_game both were dropped as well because they were empty columns
#I also believe that GamePSID no longer has any bearing on our data, so I dropped it too. 
#I also decided to drop any data that's missing the Genre, as this will be one of our main categories we look at. 
#I will create an optional csv we can use that drops the Medium category, but for now, I will keep it.

<class 'pandas.core.frame.DataFrame'>
Index: 8890 entries, 0 to 9684
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           8890 non-null   int64  
 1   GameName     8890 non-null   object 
 2   Publisher    8871 non-null   object 
 3   ReleaseYear  8867 non-null   float64
 4   Developer    8890 non-null   object 
 5   Genre        8890 non-null   object 
 6   Medium       8840 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 555.6+ KB


,ID,GameName,Publisher,ReleaseYear,Developer,Genre,Medium
count,8890.000000,8890,8871,8867.000000,8890,8890,8840
unique,NaN,8887,1713,NaN,3090,482,3
top,NaN,Prince of Persia: The Sands of Time Remake,eastasiasoft,NaN,Hamster,Platformer,Digital only
freq,NaN,3,496,NaN,124,735,5853
mean,4865.638245,NaN,NaN,2018.491485,NaN,NaN,NaN
std,2787.718850,NaN,NaN,2.008838,NaN,NaN,NaN
min,1.000000,NaN,NaN,2009.000000,NaN,NaN,NaN
25%,2474.250000,NaN,NaN,2017.000000,NaN,NaN,NaN
50%,4863.500000,NaN,NaN,2019.000000,NaN,NaN,NaN
75%,7278.750000,NaN,NaN,2020.000000,NaN,NaN,NaN


Looking at Genres:

In [44]:
#dropped_data_df['Genre'] = dropped_data_df['Genre'].str.strip().str.lower()
#dropped_data_df['Genre'].nunique()
#Running the above code, shows that there are no issues of any genres being counted as a new genre in the case of case sensitivity or have an extra space in the entry
#Nothing to drop, will take a look at the counts and such
dropped_data_df['Genre'].value_counts().sort_index()
#This shows us that the "unique"-ness of the data isn't very true, as some of these entries have multiple genres implemented into them.

Genre
Action                                             559
Action Horror                                       13
Action Horror, Action-Adventure                      1
Action Horror, First Person Shooter                 17
Action Horror, First Person Shooter, Platformer      1
                                                  ... 
Vehicular Combat                                     1
Vehicular Combat, Shoot 'em up                       1
Visual Novel                                       393
Volleyball, Sports                                   4
Volleyball, Sports, Football, Tennis                 2
Name: count, Length: 482, dtype: int64

In [45]:
#Let's try and find the truly unique genres here by finding the entries that don't have a separator, or comma
unique_genres = dropped_data_df[dropped_data_df['Genre'].str.contains(",", na=False) == False]


print(unique_genres['Genre'].describe())
#This narrows us down to 45 unique genres! That's much more realistic, still a large number though.
print(unique_genres['Genre'].unique())
#These all seem to be very unique! No overlap in genres that'd be due to a typo. 
#I haven't, but we could also use alias mapping to reduce the genre number to get even less columns here, but I think it's fine? They seem unique enough, but I could see potential overlap


count           5368
unique            45
top       Platformer
freq             735
Name: Genre, dtype: object
['Action' 'First Person Shooter' 'Stealth' 'Platformer' 'Action-Adventure'
 'Party' 'Educational & Trivia' 'Collectable Card Game' "Shoot 'em up"
 'Puzzle' 'Adventure' 'Fighting' "Beat 'em up" 'Hack & Slash' 'Strategy'
 'Sports' 'Visual Novel' 'Music' 'Simulation' 'Run & Gun'
 'Survival Horror' 'Roguelite' 'Action Horror' 'Management'
 'Third Person Shooter' 'Arcade Racing' 'Dungeon Crawler' 'Card & Board'
 'Simulation Racing' 'Tower Defence' 'Sandbox' 'Collection' 'Casino'
 'Point & Click' 'Survival' 'Pinball' 'Role Playing' 'MOBA' 'Dance' 'Mech'
 'Battle Royale' 'On Rails' 'Metroidvania' 'Open World' 'Vehicular Combat']


In [46]:
#OneHotEncoder time, or perhaps MultiLabelBinarizer would work well? I will look into this more ASAP
ohe = OneHotEncoder(handle_unknown = 'ignore')
